In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]

load gdsc1


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)

    val_labels = sampler.test_data[sampler.test_mask]

    if len(np.unique(val_labels)) < 2:
        print(f"Target {target_index} skipped: Validation set has only one class.")
        return None, None

    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
from joblib import Parallel, delayed
from tqdm import tqdm

n_kfold = 1
n_jobs = 50  # 並列数


def process_iteration(dim, target_index, seed, args):
    """各反復処理をカプセル化した関数"""
    if dim:
        if drug_sum[target_index] < 10:
            return None, None
    else:
        if cell_sum[target_index] < 10:
            return None, None

    fold_results = []
    for fold in range(n_kfold):
        true_data, predict_data = nihgcn_new(
            cell_exprs=exprs,
            drug_finger=drug_finger,
            res_mat=res,
            null_mask=null_mask,
            target_dim=dim,
            target_index=target_index,
            evaluate_fun=roc_auc,
            args=args,
            seed=seed,
        )
        fold_results.append((true_data, predict_data))

    return fold_results


# 並列処理の実行
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()

for dim in target_dim:
    # 全タスクを事前に生成
    tasks = [
        (dim, target_index, seed, args)
        for seed, target_index in enumerate(np.arange(res.shape[dim]))
    ]

    # 並列実行（プログレスバー付き）
    results = Parallel(n_jobs=n_jobs, verbose=0, prefer="threads")(
        delayed(process_iteration)(*task)
        for task in tqdm(tasks, desc=f"Processing dim {dim}")
    )

    # 結果の結合
    for fold_results in results:
        if fold_results is None:
            continue
        for true_data, predict_data in fold_results:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)],
                ignore_index=True,
                copy=False,  # メモリ節約のため
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)],
                ignore_index=True,
                copy=False,
            )

  0%|          | 0/331 [00:00<?, ?it/s]

epoch:   0 loss:0.699242 auc:0.4583
epoch:  20 loss:0.197577 auc:0.7171
epoch:  40 loss:0.178471 auc:0.7631
epoch:  60 loss:0.165197 auc:0.7529
epoch:  80 loss:0.155157 auc:0.7508
epoch: 100 loss:0.145307 auc:0.7706
epoch: 120 loss:0.136342 auc:0.7684
epoch: 140 loss:0.128480 auc:0.7533
epoch: 160 loss:0.122975 auc:0.7514
epoch: 180 loss:0.119942 auc:0.7449
epoch: 200 loss:0.117033 auc:0.7382
epoch: 220 loss:0.114759 auc:0.7506
epoch: 240 loss:0.112547 auc:0.7543
epoch: 260 loss:0.115715 auc:0.7574
epoch: 280 loss:0.110208 auc:0.7524
epoch: 300 loss:0.108950 auc:0.7426
epoch: 320 loss:0.109509 auc:0.7385
epoch: 340 loss:0.107695 auc:0.7395
epoch: 360 loss:0.111029 auc:0.7363
epoch: 380 loss:0.107267 auc:0.7396
epoch: 400 loss:0.106523 auc:0.7393
epoch: 420 loss:0.106578 auc:0.7248
epoch: 440 loss:0.107103 auc:0.7304
epoch: 460 loss:0.105830 auc:0.7366
epoch: 480 loss:0.106754 auc:0.7294
epoch: 500 loss:0.105477 auc:0.7439
epoch: 520 loss:0.105351 auc:0.7329
epoch: 540 loss:0.106867 auc

  0%|          | 1/331 [00:43<4:01:08, 43.84s/it]

Fit finished.
epoch:   0 loss:0.698236 auc:0.4626
epoch:  20 loss:0.197234 auc:0.7867
epoch:  40 loss:0.177293 auc:0.8144
epoch:  60 loss:0.164366 auc:0.8116
epoch:  80 loss:0.154537 auc:0.7701
epoch: 100 loss:0.145010 auc:0.7452
epoch: 120 loss:0.136741 auc:0.7368
epoch: 140 loss:0.128169 auc:0.7341
epoch: 160 loss:0.122844 auc:0.7479
epoch: 180 loss:0.119499 auc:0.7341
epoch: 200 loss:0.116975 auc:0.7452
epoch: 220 loss:0.114248 auc:0.7396
epoch: 240 loss:0.113895 auc:0.7202
epoch: 260 loss:0.111589 auc:0.7590
epoch: 280 loss:0.110825 auc:0.7202
epoch: 300 loss:0.110594 auc:0.7729
epoch: 320 loss:0.109576 auc:0.7701
epoch: 340 loss:0.108540 auc:0.7341
epoch: 360 loss:0.109465 auc:0.7424
epoch: 380 loss:0.107715 auc:0.7285
epoch: 400 loss:0.108089 auc:0.6981
epoch: 420 loss:0.107017 auc:0.7618
epoch: 440 loss:0.107831 auc:0.7452
epoch: 460 loss:0.107103 auc:0.7452
epoch: 480 loss:0.106248 auc:0.7452
epoch: 500 loss:0.110417 auc:0.6731
epoch: 520 loss:0.106598 auc:0.7230
epoch: 540 los

  1%|          | 2/331 [01:24<3:51:20, 42.19s/it]

...

Fit finished.
epoch:   0 loss:0.693977 auc:0.4575
epoch:  20 loss:0.199438 auc:0.6125
epoch:  40 loss:0.179225 auc:0.7500
epoch:  60 loss:0.165934 auc:0.7050
epoch:  80 loss:0.156386 auc:0.6850
epoch: 100 loss:0.146781 auc:0.6950
epoch: 120 loss:0.137493 auc:0.6800
epoch: 140 loss:0.131985 auc:0.6975
epoch: 160 loss:0.124255 auc:0.7450
epoch: 180 loss:0.120058 auc:0.7475
epoch: 200 loss:0.120494 auc:0.6825
epoch: 220 loss:0.114852 auc:0.7525
epoch: 240 loss:0.113342 auc:0.7425
epoch: 260 loss:0.111697 auc:0.7350
epoch: 280 loss:0.111973 auc:0.7450
epoch: 300 loss:0.109856 auc:0.7175
epoch: 320 loss:0.111591 auc:0.7175
epoch: 340 loss:0.108764 auc:0.7200
epoch: 360 loss:0.108327 auc:0.7150
epoch: 380 loss:0.111027 auc:0.7450
epoch: 400 loss:0.107918 auc:0.6775
epoch: 420 loss:0.106978 auc:0.6975
epoch: 440 loss:0.111247 auc:0.7075
epoch: 460 loss:0.106728 auc:0.6675
epoch: 480 loss:0.107081 auc:0.7200
epoch: 500 loss:0.106455 auc:0.6975
epoch: 520 loss:0.105871 auc:0.7050
epoch: 540 los

 11%|█▏        | 38/331 [29:16<4:47:53, 58.95s/it]

Fit finished.


 11%|█▏        | 38/331 [29:16<3:45:46, 46.23s/it]


KeyboardInterrupt: 

In [ ]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")